# Steady Navier-Stokes equations

We consider the stationary Navier-Stokes problem defined in the unit square domain $\Omega := {[0,1]}^2$
$$
\begin{cases}
  ({\boldsymbol{u}}\cdot\boldsymbol{\nabla})\ \boldsymbol{u}
  -\dfrac{1}{\text{Re}}\boldsymbol{\Delta} \boldsymbol{u} + \boldsymbol{\nabla} p = 0, 
  \qquad&\text{in }\Omega, \\
  \boldsymbol{\nabla} \cdot \boldsymbol{u} = 0, 
  \qquad &\text{in }\Omega, \\
  \boldsymbol{u} = 1\boldsymbol{i} + 0\boldsymbol{j}, \qquad&\text{on }\Gamma^{\text{up}} = \{ 0 \le x \le 1, \, y=1 \}, \\
  \boldsymbol{u} = \boldsymbol{0}, \qquad&\text{on }\partial\Omega \backslash \Gamma^{\text{up}}.
\end{cases}
$$

## Numerical approximation with FEniCS

In [ ]:
%%capture
try:
    import dolfin
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/fenics-install.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"
    import dolfin

from fenics import *
import numpy as np

### Taylor-Hood discretization

Using the stable pair of spaces $\mathbb{P}^2/\mathbb{P}^1$ solve the problem using the **fixed-point iteration method**. Consider a stopping criterion based on the relative increment
$$
\frac{||\boldsymbol{u}_k-\boldsymbol{u}_{k-1}||_{H^1}}{||\boldsymbol{u}_{k-1}||_{H^1}}
+\frac{||\boldsymbol{p}_k-\boldsymbol{p}_{k-1}||_{L^2}}{||\boldsymbol{p}_{k-1}||_{L^2}}
\le 10^{-6}.
$$

*   Try to solve the problem with different values of Reynolds number ($\text{Re}=3,30,300,3000$). 
*   Assess the convergence performance of the iterative method.

In [ ]:
# 1. mesh generation
nx, ny = 20, 20
mesh = UnitSquareMesh(nx, ny, 'crossed')

# 2. finite element space
degree = 1
V = VectorElement('CG', mesh.ufl_cell(), degree + 1)
Q = FiniteElement('CG', mesh.ufl_cell(), degree)

X = FunctionSpace(mesh, MixedElement([V, Q]))

u_boundary = Expression((
        'near(x[1], 1) ? 1.0 : 0.0',
        '0'
    ), degree=0)

def boundary(x, on_boundary):
  return on_boundary

def origin(x, on_boundary):
  return near(x[0], 0) and near(x[1], 0)

bc = [DirichletBC(X.sub(0), u_boundary, boundary),
      DirichletBC(X.sub(1), Constant(0), origin, 'pointwise')]

In [ ]:
# 3. problem definition
def solve_linear(X, advection, Re, f, bc):
  u, p = TrialFunctions(X)
  v, q = TestFunctions(X)

  a = (inner(grad(u), grad(v)) / Re - p * div(v) + div(u) * q) * dx
  L = dot(f, v) * dx
  if advection:
    a += dot(grad(u) * advection, v) * dx
  
  x = Function(X)
  solve(a == L, x, bc)
  
  return x.split()

In [ ]:
# 4. solution
Re = Constant(3000)
f = Constant((0, 0))

uh, ph = solve_linear(X, None, Re, f, bc)

niter = 100
tolerance = 1e-6
for i in range(niter):
  uh_old, ph_old = uh, ph
  uh, ph = solve_linear(X, uh_old, Re, f, bc)
  
  error = (errornorm(uh, uh_old, 'H1') / norm(uh_old, 'H1') +
           errornorm(ph, ph_old, 'L2') / norm(ph_old, 'L2'))
  print('step {}: {:.3e}'.format(i, error))
  if error < tolerance:
    break

uh.rename('velocity', 'velocity')
ph.rename('pressure', 'pressure')

File('velocity.pvd') << uh
File('pressure.pvd') << ph

### SUPG stabilized FEM

We introduce the SUPG stabilization for the solution of the
linearized problem. Use the following stabilization parameter
$$
\delta_K = \frac{h_K}{2 ||{\boldsymbol{u}}||} \xi(\mathbb{P}e_K),
$$
 where the local Péclet number is defined as
$$
\mathcal{Pe}_K := \frac{||{\boldsymbol{u}}||h_K}{2} \text{Re},
$$
and $\xi$ is the *upwind* function
$$
      \xi(\theta) = \min{(1,\theta)}.
$$
Compare the solutions of the previous point with the solutions obtained with the SUPG stabilization.

In [ ]:
# 3. problem definition
def solve_linear(X, advection, Re, f, bc):
  u, p = TrialFunctions(X)
  v, q = TestFunctions(X)
  
  a = (inner(grad(u), grad(v)) / Re - p * div(v) + div(u) * q) * dx
  L = dot(f, v) * dx
  if advection:
    a += dot(grad(u) * advection, v) * dx

    A = lambda u, p: -div(grad(u)) / Re + (grad(u) * advection) + grad(p)
    A_SS = lambda u, p: (grad(u) * advection) + div(advection) * u / 2 + grad(p)
  
    h = CellDiameter(X.mesh())
    anorm = sqrt(dot(advection, advection))
    tau_K = 0.5 * h / conditional(anorm * h * Re > 2, anorm, 2 / h / Re)
  
    a += tau_K * (dot(A(u, p), A_SS(v, q)) + div(u) * div(v)) * dx
    L += tau_K * (dot(f, A_SS(v, q))) * dx
  
  x = Function(X)
  solve(a == L, x, bc)
  
  return x.split()

In [ ]:
# 4. solution
Re = Constant(20000)
f = Constant((0, 0))

uh, ph = solve_linear(X, Constant((0, 0)), Re, f, bc)

niter = 100
tolerance = 1e-6
for i in range(niter):
  uh_old, ph_old = uh, ph
  uh, ph = solve_linear(X, uh_old, Re, f, bc)
  
  error = (errornorm(uh, uh_old, 'H1') / norm(uh_old, 'H1') +
           errornorm(ph, ph_old, 'L2') / norm(ph_old, 'L2'))
  print('step {}: {:.3e}'.format(i, error))
  if error < tolerance:
    break

uh.rename('velocity', 'velocity')
ph.rename('pressure', 'pressure')

File('velocity_stab.pvd') << uh
File('pressure_stab.pvd') << ph

step 0: 2.148e+01
step 1: 1.058e+00
step 2: 1.286e+00
step 3: 1.061e+00
step 4: 8.802e-01
step 5: 1.490e+00
step 6: 8.893e-01
step 7: 9.111e-01
step 8: 4.157e-01
step 9: 5.192e-01
step 10: 4.127e-01
step 11: 4.274e-01
step 12: 3.657e-01
step 13: 4.121e-01
step 14: 3.375e-01
step 15: 3.953e-01
step 16: 2.887e-01
step 17: 3.503e-01
step 18: 2.817e-01
step 19: 4.328e-01
step 20: 2.520e-01
step 21: 5.042e-01
step 22: 1.378e-01
step 23: 2.766e-01
step 24: 4.838e-02
step 25: 5.878e-02
step 26: 2.189e-02
step 27: 3.053e-02
step 28: 1.479e-02
step 29: 1.703e-02
step 30: 8.573e-03
step 31: 9.353e-03
step 32: 4.490e-03
step 33: 4.899e-03
step 34: 2.230e-03
step 35: 2.458e-03
step 36: 1.086e-03
step 37: 1.244e-03
step 38: 5.508e-04
step 39: 6.445e-04
step 40: 2.844e-04
step 41: 3.374e-04
step 42: 1.487e-04
step 43: 1.782e-04
step 44: 7.831e-05
step 45: 9.386e-05
step 46: 4.047e-05
step 47: 4.900e-05
step 48: 2.086e-05
step 49: 2.551e-05
step 50: 1.061e-05
step 51: 1.322e-05
step 52: 5.422e-06
ste

#### **Exercise**: Repeat the exercise using the pair of spaces $\mathbb{P}^1/\mathbb{P}^1$.